# Preprocessing of the trade network

In [1]:
def net_symmetrisation(wtn_file, exclude_countries):
    DG=nx.DiGraph()
    
    Reporter_pos=1
    Partner_pos=3
    Flow_code_pos=2
    Value_pos=9
    
    dic_trade_flows={}
    hfile=open(wtn_file,'r')
    
    header=hfile.readline()
    lines=hfile.readlines()
    for l in lines:
        l_split=l.split(',')
        #the following is to prevent parsing lines without data
        if len(l_split)<2: continue 
        reporter=int(l_split[Reporter_pos])
        partner=int(l_split[Partner_pos])
        flow_code=int(l_split[Flow_code_pos])
        value=float(l_split[Value_pos])
        
        if ( (reporter in exclude_countries) or \
            (partner in exclude_countries) or (reporter==partner) ): 
            continue
        
        if flow_code==1 and value>0.0:
            #1=Import, 2=Export
            if (partner,reporter,2) in dic_trade_flows: 
                DG[partner][reporter]['weight']= \
                (DG[partner][reporter]['weight']+value)/2.0
            else:
                DG.add_edge(partner, reporter, weight=value)
                dic_trade_flows[(partner,reporter,1)]= \
                value #this is to mark the exixtence of the link
        
        elif flow_code==2 and value>0.0:
            #1=Import, 2=Export
            if (reporter,partner,1) in dic_trade_flows: 
                DG[reporter][partner]['weight']= \
                (DG[reporter][partner]['weight']+value)/2.0
            else:
                DG.add_edge(reporter, partner, weight=value)
                #this is to mark the exixtence of the link
                dic_trade_flows[(reporter,partner,2)]=value 
        else:
            print("trade flow not present\n")
        
    hfile.close()
    
    return DG

In [6]:
import networkx as nx
#countries to be excluded
exclude_countries=[472,899,471,129,221,97,697,492,838,473,536, 637,290,527,577,490,568,636,839,879,0]
DG = net_symmetrisation("../Data/Comtrade/comtrade_trade_data_total_2003.csv", exclude_countries)

In [7]:
#import the country name
import csv

country_dict = {}

with open("../Data/Comtrade/Country Code and Name ISO2 ISO3_annotated.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        country_dict[int(row[0])] =  row[1]

In [12]:
for node in DG.nodes():
    DG.node[node]["name"] = country_dict[node]

In [13]:
nx.write_gpickle(DG, "../Data/Comtrade/comtrade.gpickle")

In [17]:
DG.nodes(data=True)

[(512, {'name': 'Oman'}),
 (4, {'name': 'Afghanistan'}),
 (598, {'name': 'Papua New Guinea'}),
 (520, {'name': 'Nauru'}),
 (10, {'name': 'Antarctica'}),
 (12, {'name': 'Algeria'}),
 (16, {'name': 'American Samoa'}),
 (92, {'name': 'Br. Virgin Isds'}),
 (530, {'name': 'Neth. Antilles'}),
 (20, {'name': 'Andorra'}),
 (716, {'name': 'Zimbabwe'}),
 (24, {'name': 'Angola'}),
 (516, {'name': 'Namibia'}),
 (28, {'name': 'Antigua and Barbuda'}),
 (31, {'name': 'Azerbaijan'}),
 (32, {'name': 'Argentina'}),
 (36, {'name': 'Australia'}),
 (40, {'name': 'Austria'}),
 (860, {'name': 'Uzbekistan'}),
 (554, {'name': 'New Zealand'}),
 (44, {'name': 'Bahamas'}),
 (690, {'name': 'Seychelles'}),
 (558, {'name': 'Nicaragua'}),
 (48, {'name': 'Bahrain'}),
 (8, {'name': 'Albania'}),
 (562, {'name': 'Niger'}),
 (51, {'name': 'Armenia'}),
 (52, {'name': 'Barbados'}),
 (566, {'name': 'Nigeria'}),
 (56, {'name': 'Belgium'}),
 (570, {'name': 'Niue'}),
 (60, {'name': 'Bermuda'}),
 (574, {'name': 'Norfolk Isds'}),

In [124]:
for s,t in DG_filtered.edges():
    if G[s][t]["weight"] > 10000000:
        Di.add_edge(s,t, weight=1.0/G[s][t]["weight"])

In [125]:
len(Di.edges())

7629

In [137]:
closeness = nx.closeness_centrality(Di, distance="weight", normalized=True)
betweeness = nx.betweenness_centrality(Di, weight="weight")

In [121]:
from collections import OrderedDict
closeness_sorted_by_value = OrderedDict(sorted(closeness.items(), key=lambda x: x[1]))

In [135]:
for k,v in closeness_sorted_by_value.items():
    print(country_dict[k],v)

Norfolk Isds 0.0
Timor-Leste 0.0
Cape Verde 0.0
Saint Helena 0.0
Saint Pierre and Miquelon 0.0
Christmas Isds 0.0
Sao Tome and Principe 0.0
Mayotte 0.0
Cook Isds 0.0
Eritrea 0.0
Gambia 0.0
Holy See (Vatican City State) 0.0
Anguilla 0.0
N. Mariana Isds 0.0
Wallis and Futuna Isds 0.0
Montserrat 0.0
Tokelau 9258082.634484231
United States Minor Outlying Islands 9683418.79552415
Grenada 9691744.531402446
Comoros 9814455.387466878
Tonga 9828506.206871413
Nauru 10471821.774959823
Dominica 11236917.559598865
Guam 11503866.45235191
San Marino 11808968.13872879
Palau 12061521.281094389
Saint Vincent and the Grenadines 12459360.412656516
Turks and Caicos Isds 13130628.291603984
American Samoa 14765303.257929271
Djibouti 16510183.439751564
Kiribati 16946880.697675165
Burundi 18852059.333993006
Saint Lucia 18877819.752256136
Chad 20420463.293426387
Solomon Isds 20473981.987956345
Bunkers 21049144.49070877
Vanuatu 22335044.06799094
Rwanda 25838357.309382237
Guinea-Bissau 28839829.421735425
Saint Ki

In [138]:
from collections import OrderedDict
betweeness_sorted_by_value = OrderedDict(sorted(betweeness.items(), key=lambda x: x[1]))

In [139]:
for k,v in betweeness_sorted_by_value.items():
    print(country_dict[k],v)

Oman 0.0
Afghanistan 0.0
Nauru 0.0
Algeria 0.0
Neth. Antilles 0.0
Andorra 0.0
Aruba 0.0
Namibia 0.0
Antigua and Barbuda 0.0
Azerbaijan 0.0
Austria 0.0
Bahamas 0.0
Seychelles 0.0
Nicaragua 0.0
Bahrain 0.0
Albania 0.0
Armenia 0.0
Barbados 0.0
Bermuda 0.0
Norfolk Isds 0.0
Bhutan 0.0
Philippines 0.0
Bolivia 0.0
United States Minor Outlying Islands 0.0
Bosnia Herzegovina 0.0
FS Micronesia 0.0
Marshall Isds 0.0
Trinidad and Tobago 0.0
Panama 0.0
Belize 0.0
Papua New Guinea 0.0
Paraguay 0.0
Solomon Isds 0.0
Br. Virgin Isds 0.0
Brunei Darussalam 0.0
American Samoa 0.0
Bulgaria 0.0
Burundi 0.0
Comoros 0.0
Guinea-Bissau 0.0
Myanmar 0.0
Timor-Leste 0.0
Cambodia 0.0
Cameroon 0.0
Qatar 0.0
Cuba 0.0
Romania 0.0
Cape Verde 0.0
Rwanda 0.0
Cayman Isds 0.0
Central African Rep. 0.0
Saint Helena 0.0
Sri Lanka 0.0
Saint Kitts and Nevis 0.0
Chad 0.0
Saint Lucia 0.0
Chile 0.0
Saint Pierre and Miquelon 0.0
Saint Vincent and the Grenadines 0.0
Belarus 0.0
Christmas Isds 0.0
Nepal 0.0
Sao Tome and Principe 0.0
